<a href="https://colab.research.google.com/github/appletreeleaf/NLP_Projects/blob/main/%5BLangChain%5D_PDF_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문서 검색 챗봇 만들기 LangChain

> 유튜브 [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

In [ ]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken

## OpenAI API Key

https://platform.openai.com/account/api-keys

In [2]:
import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import os

os.environ["OPENAI_API_KEY"] = ""

In [8]:
!wget https://www.holybooks.com/wp-content/uploads/Demian-By-Hermann-Hesse.pdf

--2024-03-01 18:12:31--  https://www.holybooks.com/wp-content/uploads/Demian-By-Hermann-Hesse.pdf
Resolving www.holybooks.com (www.holybooks.com)... 46.4.115.122
Connecting to www.holybooks.com (www.holybooks.com)|46.4.115.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5683449 (5.4M) [application/pdf]
Saving to: ‘Demian-By-Hermann-Hesse.pdf’

Demian-By-Hermann-H 100%[===================>]   5.42M  3.79MB/s    in 1.4s    

2024-03-01 18:12:34 (3.79 MB/s) - ‘Demian-By-Hermann-Hesse.pdf’ saved [5683449/5683449]



## Preprocess a PDF file

In [11]:
reader = PdfReader("Demian-By-Hermann-Hesse.pdf")
print(reader)
raw_text = ""

# pdf파일의 페이지를 순회하며 하나의 문자열로 추출함

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text[:1000]

"DEMIAN • \nDownloaded from https://www.holybooks.comHERMANN \nHESSE • DEMIAN \n* \nTranslated by W. J. Strachan \nLondon \nDownloaded from https://www.holybooks.comPrologue \nI cannot tell my story without going a long way back. \nIf it were possible I would go back much farther still to \nthe very earliest years of my childhood and beyond them \nto my family origins. \nWhen poets write novels they are apt to behave as if \nthey were gods, with the power to look beyond and com\xad\nprehend any human story and serve it up as if the \nAlmighty himself, omnipresent, were relating it in all \nits naked truth. That I am no more able to do than the \npoets. But my story is more important to me than any \npoet's story to him, for it is my own-and it is the story \nof a huffian being-not an invented, idealised person \nbut a real, live, uniq:-e being. What constitutes a real, \nlive human being is more of a mystery than ever these \ndays, and men-each one of whom is a valuable, unique \nexper

## Summarize 요약

In [12]:
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0) # gpt-3
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


```
요약은 비교적 단순한 task이므로
추론비용을 줄이기 위해
gpt-3로도 충분합니다.
```

In [13]:
summarize_document_chain.run(raw_text)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' "Demian" by Hermann Hesse is a novel about a young man\'s journey of self-discovery as he reflects on his past and encounters influential figures like Franz Kromer and Max Demian. He struggles with inner turmoil and explores ideas of good and evil, eventually finding love and a sense of purpose. The story also touches on themes of individual growth, spirituality, and the impending change of an old world.'

### How much?

- text-davinci, 6 requests
- 101,611 prompt + 8,930 completion = 110,541 tokens
- $0.47 == ₩611

## Question Answering 질문 답변

1~3분 소요

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo") # gpt-3.5-turbo, gpt-4

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [15]:
qa_document_chain.run(
    input_document=raw_text,
    question="싱클레어를 괴롭힌 사람은?")

'Franz Kromer입니다.'

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="싱클레어는 데미안을 어디서 어떻게 만났지?")

'제공된 텍스트에서는 싱클레어가 데미안을 만난 경위에 대한 구체적인 정보가 제공되지 않습니다. 따라서 이 질문에 대한 답변을 제공할 수 없습니다.'

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="데미안의 외모를 묘사해봐")

'저는 주어진 텍스트에서 데미안의 외모에 대한 구체적인 묘사를 찾을 수 없었습니다. 따라서 데미안의 외모에 대한 정보를 제공할 수 없습니다.'

### GPT-4

11분 소요

- gpt-4-0314, 28 requests
    - 29,503 prompt + 1,111 completion = 30,614 tokens tokens
- gpt-4-0314, 42 requests
    - 44,041 prompt + 1,784 completion = 45,825 tokens
- gpt-4-0314, 10 requests
    - 10,652 prompt + 865 completion = 11,517 tokens


87,956 tokens total == $5.28

> $0.06 / 1K tokens

In [ ]:
model = ChatOpenAI(model="gpt-4") # gpt-3.5-turbo, gpt-4

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

qa_document_chain.run(
    input_document=raw_text,
    question="데미안의 외모를 묘사해봐")

'데미안의 외모는 다음과 같이 묘사되어 있습니다: 근접한 갈색 머리카락, 반여성적인 입, 이상한 밝기를 가진 강한 이마, 그리고 넓게 벌어진 녹색 눈동자로, 오른쪽 눈이 왼쪽 눈보다 약간 높게 위치해 있습니다. 또한, 데미안은 군복을 입고 은색 회색 망토를 입은 모습이 기묘하고 이상한 것처럼 묘사되어 있습니다.'

```
앞서 gpt-3에서는 적절한 대답을 하지 못한 것과 별개로
gpt-4에서는 데미안의 외모에 대한 자세한 묘사가 포함되어 있다.
```

## 엑셀, CSV 검색, Aggregation

In [16]:
import pandas as pd

df = pd.read_csv("https://github.com/kairess/toy-datasets/raw/master/titanic.csv")

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
!pip install langchain_experimental

In [21]:
from langchain_experimental.agents import create_pandas_dataframe_agent
llm = OpenAI(temperature=0)
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


'891'

In [22]:
agent.run("행이 몇 개지?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


'891'

```
동일한 질문을 영어와 한국어로 실행한 결과
동일한 답변을 얻을 수 있었다.
이는

In [ ]:
agent.run("승객들의 평균 연령은?")



> Entering new AgentExecutor chain...
Thought: 나는 이 데이터 프레임에서 평균 연령을 찾아야한다.
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought: 나는 이제 최종 답을 알게되었다.
Final Answer: 승객들의 평균 연령은 29.7세입니다.

> Finished chain.


'승객들의 평균 연령은 29.7세입니다.'

In [ ]:
agent.run("남성과 여성의 비율은?")



> Entering new AgentExecutor chain...
Thought: 남성과 여성의 수를 세야한다.
Action: python_repl_ast
Action Input: df['Sex'].value_counts()
Observation: male      577
female    314
Name: Sex, dtype: int64
Thought: 남성과 여성의 비율을 계산해야한다.
Action: python_repl_ast
Action Input: df['Sex'].value_counts() / df['Sex'].count()
Observation: male      0.647587
female    0.352413
Name: Sex, dtype: float64
Thought: 남성과 여성의 비율을 알았다.
Final Answer: 남성의 비율은 0.647587, 여성의 비율은 0.352413입니다.

> Finished chain.


'남성의 비율은 0.647587, 여성의 비율은 0.352413입니다.'

In [ ]:
agent.run("객실 등급과 성별에 따른 생존자 수를 계산해줘")



> Entering new AgentExecutor chain...
Thought: 먼저 객실 등급과 성별로 그룹화해야 할 것 같다.
Action: python_repl_ast
Action Input: df.groupby(['Pclass', 'Sex'])['Survived'].sum()
Observation: Pclass  Sex   
1       female    91
        male      45
2       female    70
        male      17
3       female    72
        male      47
Name: Survived, dtype: int64
Thought: 이제 생존자 수를 계산할 수 있다.
Final Answer: 객실 등급과 성별에 따른 생존자 수는 각각 91명, 45명, 70명, 17명, 72명, 47명입니다.

> Finished chain.


'객실 등급과 성별에 따른 생존자 수는 각각 91명, 45명, 70명, 17명, 72명, 47명입니다.'